# Content based Filtering
In this approach I'll use some properties and the metadata of a particular item to suggest other items with similar characteristics.

Content based filtering always provide existent item to make raccomandation contrary to collaborative filtering which is based to the attitudes of costumers.

In this section I'll calculate pairwise cosine similarity scores for all beers  based on their Beer_style (that is a category) and Beer_ABV (alcholic grade) then recommend beers with the highest similarity scores to the user’s input.

I'm also using Natural Language processing to understand and suggest similar movies to a user’s input. 


Firstly we recognize the most relevant features for a beer.

Beer_style and Beer_ABV because for me are the most relevant feature.
I'm going to merge them in unique column, vectorize this column and then compute cosine similarity.

I have to vectorized the word merged throught Frequency-Inverse Document Frequency (TF-IDF) vectors. This give a matrix where each column represents a word in the description vocabulary and each row represents a Beer.

## Cosine Similarity 
Cosine similarity is a mathematical computation that tells us the similarity between two vectors A and B. In effect, I'm  calculating the cosine of the angle theta between these two vectors. The function returns a value between -1, indicating complete opposite vectors, to 1, indicating the same vector. 0 indicates a lack of correlation between the vectors, and intermediate values indicate intermediate levels of similarity.

Tha main problem with using  cosine similarity is the  computation between each possible pair of movies. If we have 1000 beers , then we need to perform 1000000 beers.
So I will decrese my dataset.
 



In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
!pip install pyspark
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark = SparkSession.builder.appName("Raccomandation").getOrCreate()
data = spark.read.csv("/content/drive/MyDrive/UNIVERSITA/BIG DATA/BIG DATA /beer_reviews.csv",inferSchema= True,header = True)

In [ ]:
dataset = data.select('review_profilename', 'beer_beerid', 'review_overall','beer_style','beer_name','beer_abv')
preprocessing_data = dataset.na.drop()

##trasform unique username in unique id 

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="review_profilename", outputCol="customer_id").fit(preprocessing_data)

In [ ]:
full_data = indexer.transform(preprocessing_data)

In [ ]:
full_data.show()

+------------------+-----------+--------------+--------------------+--------------------+--------+-----------+
|review_profilename|beer_beerid|review_overall|          beer_style|           beer_name|beer_abv|customer_id|
+------------------+-----------+--------------+--------------------+--------------------+--------+-----------+
|           stcules|      47986|           1.5|          Hefeweizen|        Sausa Weizen|     5.0|       85.0|
|           stcules|      48213|           3.0|  English Strong Ale|            Red Moon|     6.2|       85.0|
|           stcules|      48215|           3.0|Foreign / Export ...|Black Horse Black...|     6.5|       85.0|
|           stcules|      47969|           3.0|     German Pilsener|          Sausa Pils|     5.0|       85.0|
|    johnmichaelsen|      64883|           4.0|American Double /...|       Cauldron DIPA|     7.7|       45.0|
|           oline73|      52159|           3.0|Herbed / Spiced Beer| Caldera Ginger Beer|     4.7|      420.0|
|

## Pre-processing
Preparing the dataset for using Natural Language processing Tokenization


In [ ]:
preprocessing_data = full_data.toPandas()

In [ ]:
data2 = full_data.toPandas()

In [ ]:
preprocessing_data.count()

review_profilename    1518493
beer_beerid           1518493
review_overall        1518493
beer_style            1518493
beer_name             1518493
beer_abv              1518493
customer_id           1518493
dtype: int64

Drop duplicates beer name.
Doesn't metter the reviews but the beer profile.

In [ ]:
preprocessing = preprocessing_data.head(15000)

In [ ]:
df = preprocessing

We are stripping all the beer style in order have one sigle value to vectorize.
Each single word now correspond to a style of beer 

In [ ]:
for ind in df.index:
     df['beer_style'][ind] = df['beer_style'][ind].replace(" ","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
preprocessing = df

##Merging the two features togheter  

In [ ]:
preprocessing['beer_abv'] = preprocessing['beer_abv'].apply(str)
preprocessing['description'] = preprocessing['beer_style'] + " " +  preprocessing['beer_abv']
preprocessing['description']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0                           Hefeweizen 5.0
1                     EnglishStrongAle 6.2
2                  Foreign/ExportStout 6.5
3                       GermanPilsener 5.0
4           AmericanDouble/ImperialIPA 7.7
                       ...                
14995    AmericanDouble/ImperialStout 10.6
14996    AmericanDouble/ImperialStout 10.6
14997    AmericanDouble/ImperialStout 10.6
14998    AmericanDouble/ImperialStout 10.6
14999    AmericanDouble/ImperialStout 10.6
Name: description, Length: 15000, dtype: object

Drop duplicates beer name, I don't need of multiple reviews I want to create a beer profile for each beers based on it's recommendation 

In [ ]:
preprocessing = preprocessing.drop_duplicates(subset="beer_name", keep='first', inplace=False, ignore_index=False) 

## Vectorize the words merged
I want to convert all description rows in a vector of words.

I used Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each description row.
This will give you a matrix where each column represents a word in the description vocabulary and each row represents a Beer.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
tf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tf.fit_transform(preprocessing['description'])

In [ ]:
tfidf_matrix.shape

(1238, 136)

## Cosine Similarity 


I use the cosine function to compute the similarity score between beers, where each beer will have a similarity score with every other beer.

Cosine similarity is a mathematical computation that tells us the similarity between two vectors. I'm  calculating the cosine of the angle theta between these two vectors. The function returns a value between -1, indicating complete opposite vectors, to 1, indicating the same vector. 0 indicates a lack of correlation between the vectors, and intermediate values indicate intermediate levels of similarity.

Tha main problem with using  cosine similarity is the  computation between each possible pair of movies. If we have 1000 beers , then we need to perform 1000000 beers.
So I will decrese my dataset.


In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


we need a mechanism to identify the index of a beer in our metadata DataFrame, given its name.

In [ ]:
preprocessing = preprocessing.reset_index()
indices = pd.Series(preprocessing.index, index=preprocessing['beer_name'])


#we need a mechanism to identify the index of a beer in our metadata DataFrame, given its name.

##Get recommandations

In [ ]:
print(indices)

beer_name
Sausa Weizen                      0
Red Moon                          1
Black Horse Black Beer            2
Sausa Pils                        3
Cauldron DIPA                     4
                               ... 
Founders CBS Imperial Stout    1233
Founders Grand Rapids Stout    1234
Founders All Day IPA           1235
Founders Vanilla Porter        1236
Founders Devil Dancer          1237
Length: 1238, dtype: int64


Get recommendations By beers_name

In [ ]:
def get_recommendations(beer_name):
   # print(cosine_sim)
    idx = indices[beer_name]
    #print(idx)
   # print(type(idx))
    sim_scores = list(enumerate(cosine_sim[idx])) # enumrate each item id with cosine similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    #print(sim_scores)
    beer_indices = [i[0] for i in sim_scores]
    #print(beer_indices)
    return indices.iloc[beer_indices]
    #beer_name.iloc[beer_indices]

In [ ]:
table = get_recommendations('Sausa Pils')

In [ ]:
print(table)

beer_name
Kirner Pils Premium             29
Caldera Pilsener Bier           30
Amstel 1870                     67
Pilsner                        127
Zahre Chiara Pilsen            465
El Dorado Premium Pilsener     555
Freiherrn Pils                 569
Export Pils                    762
Kaiser Pils, Feinherb          766
German Pilsener                842
Thomaskirch Pils               889
Sternquell Pils                898
Hartmann Edelpils              960
MM Pils                       1048
Sausa Weizen                     0
Red Moon                         1
Black Horse Black Beer           2
Cauldron DIPA                    4
Caldera Ginger Beer              5
Amstel Light                     6
Caldera Oatmeal Stout            7
Caldera OBF 15                   8
Rauch Ür Bock                    9
Caldera Pale Ale                10
Pilot Rock Porter               11
Vas Deferens Ale                12
Caldera IPA                     13
Old Growth Imperial Stout       14
Dry Hop Re

## Precision & Recall evaluation
RMSE work only if we have a predict ratings ( is the error between the real rating and the prediction)
So there are other ways to measure the accuracy besides RMSE. In that case i think about using  precision and recall to evaluate your recommendations.

There are multiple ways to evaluate this content based system with precision and recall.
It always depends to the criteria you decide the relevant and irrelevant prediction.


For exemple my idea is to take  30 seggested beers by one user and see if this user have already drunk these beers (before the prediction).
If both thinks appened there is a True Positive while if a beer suggested is not in the list of viewed is a false positive.
While if a rated beer is not in suggested is a false negative and so on.

NB : This is not the only one solution and this couldn't work for other system and also for mine because evaluating content based is almost difficult.





return a list containing tuple (customer,beer) that are the beers rated five stars by customers

In [ ]:
#select a costumer that like a beer
def findcustomer():
  five_star = []
  for ind in df.index:
    if df['review_profilename'][ind] == "northyorksammy":
      if df['review_overall'][ind] == 5:
            five_star.append((df["beer_name"][ind]))
  return(five_star)
beer_fivestar = findcustomer()
beer_fivestar



['B.O.R.I.S. The Crusher Oatmeal-Imperial Stout']

In [ ]:
get_recommendations('B.O.R.I.S. The Crusher Oatmeal-Imperial Stout')

beer_name
Barrel Aged B.O.R.I.S. Oatmeal Imperial Stout       33
Brouemont Russian Imperial Stout                   213
B.O.R.I.S. The Crusher Oatmeal-Imperial Stout      282
Undertow                                           316
B.O.R.I.S. #100                                    366
BORIS Royale                                       372
Cafe BORIS                                         379
Jason Green's Russian Imperial Stout               406
Speedway Imperial Stout                           1062
Broad Ripple Evilsizors Russian Imperial Stout     125
Black River Imperial Stout                         174
D.O.R.I.S. The Destroyer Double Imperial Stout     355
YouEnjoyMyStout                                    377
Sausa Weizen                                         0
Red Moon                                             1
Black Horse Black Beer                               2
Sausa Pils                                           3
Cauldron DIPA                                        4


In [ ]:
customer_925 =  get_recommendations(findcustomer()[0])
print(customer_925)

beer_name
Barrel Aged B.O.R.I.S. Oatmeal Imperial Stout       33
Brouemont Russian Imperial Stout                   213
B.O.R.I.S. The Crusher Oatmeal-Imperial Stout      282
Undertow                                           316
B.O.R.I.S. #100                                    366
BORIS Royale                                       372
Cafe BORIS                                         379
Jason Green's Russian Imperial Stout               406
Speedway Imperial Stout                           1062
Broad Ripple Evilsizors Russian Imperial Stout     125
Black River Imperial Stout                         174
D.O.R.I.S. The Destroyer Double Imperial Stout     355
YouEnjoyMyStout                                    377
Sausa Weizen                                         0
Red Moon                                             1
Black Horse Black Beer                               2
Sausa Pils                                           3
Cauldron DIPA                                        4


Get The list of all predicted beers

In [ ]:
listpredicted = []
for x in customer_925.keys():
  listpredicted.append(x)
print(listpredicted)
len(listpredicted)

['Barrel Aged B.O.R.I.S. Oatmeal Imperial Stout', 'Brouemont Russian Imperial Stout', 'B.O.R.I.S. The Crusher Oatmeal-Imperial Stout', 'Undertow', 'B.O.R.I.S. #100', 'BORIS Royale', 'Cafe BORIS', "Jason Green's Russian Imperial Stout", 'Speedway Imperial Stout', 'Broad Ripple Evilsizors Russian Imperial Stout', 'Black River Imperial Stout', 'D.O.R.I.S. The Destroyer Double Imperial Stout', 'YouEnjoyMyStout', 'Sausa Weizen', 'Red Moon', 'Black Horse Black Beer', 'Sausa Pils', 'Cauldron DIPA', 'Caldera Ginger Beer', 'Amstel Light', 'Caldera Oatmeal Stout', 'Caldera OBF 15', 'Rauch Ür Bock', 'Caldera Pale Ale', 'Pilot Rock Porter', 'Vas Deferens Ale', 'Caldera IPA', 'Dry Hop Red', 'Southside Strong', 'Caldera Märzen']


30

This function loop the original dataset searching if a beers rated by the customer 2newyorksammy" is in predicted beers

In [ ]:
def prrc(lista):
  positive_list = []
  negative_list = []
  count = 0 
  for ind in df.index:
      if data2['review_profilename'][ind] == "northyorksammy": 
        count = count +1
        if  data2['beer_name'][ind] in lista and  data2['beer_name'][ind] not in positive_list:  # if the customer has reviewed the beer and it is in the predicted list 
          positive_list.append(data2['beer_name'][ind])
        if  data2['beer_name'][ind] not in lista and data2['beer_name'][ind] not in negative_list:                      # if the customer has reviewed the beer and it is not in the predicted list
          negative_list.append(data2['beer_name'][ind])
  TP = len(positive_list)
  FN = len(negative_list)
  FP = 0
  for x in lista : 
    if x not in positive_list and x not in negative_list: 
      FP = FP + 1
  
  Precision = TP/(TP+FP)
  Recall = TP/(TP+FN)

  return (Precision,Recall) 


  

        



In [ ]:
prrc(listpredicted)

(0.2, 0.06818181818181818)

##Evaluation Result 
As already saw It's very difficult to create an evaluation methods for Content based model
Infact the method used below doesn't work that well.
Probably because there are alot of (beers more than 40000) and it's difficult that a customer review the same beers of the prediction.

Bibliography
[Getting started with Movie Reccomandations](https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system)